In [ ]:
import scanpy as sc
import torch
import sys
sys.path.append('./train')
from train.model_interface import MInterface

/root/anaconda3/envs/flash/lib/python3.9/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/root/anaconda3/envs/flash/lib/python3.9/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you m

In [2]:
# define the hypeparameters and create the model

hyperparams = {'topk_gene': 200, \
                'gene_num': 60537, \
                'vq_space': 12, \
                'hidden_dim': 128, \
                'levels': [12], \
                'condition_layer': 6, \
                'layers': 1, \
                'latent_dim': 32, \
                'gene_num': 60537, \
                'cell_communication': False, \
                'res_dir': 'train/results', \
                'ex_name': 'test', \
                'steps_per_epoch': 0}
model_path = 'pretrained_models/checkpoint.pt'
model = MInterface(**hyperparams)

params = torch.load(model_path)
params = {key.replace('_forward_module.','').replace('encoder1', 'encoder'):val for key, val in params.items()}
model.load_state_dict(params, strict=False)
model.cuda()
model.eval()

MInterface(
  (model): HomoGraph(
    (cell_emb): Linear(in_features=60537, out_features=128, bias=True)
    (gene_emb): Embedding(60537, 128)
    (expression_emb): AutoDiscretizationEmbedding2(
      (mlp): Linear(in_features=1, out_features=12, bias=True)
      (mlp2): Linear(in_features=12, out_features=12, bias=True)
      (LeakyReLU): LeakyReLU(negative_slope=0.1)
      (Softmax): Softmax(dim=-1)
      (emb): Embedding(12, 128)
      (emb_pad): Embedding(1, 128)
    )
    (out): Linear(in_features=128, out_features=60537, bias=True)
    (feature_emb): Embedding(42384, 128, padding_idx=1)
    (pos_emb): Embedding(20, 128)
    (encoder): CellGraph(
      (transformer_blocks): ModuleList(
        (0): TransformerBlock(
          (mhsa): MultiHeadSelfAttention(
            (to_qvk): Linear(in_features=128, out_features=384, bias=False)
            (W_0): Linear(in_features=128, out_features=128, bias=False)
          )
          (drop): Dropout(p=0.1, inplace=False)
          (norm_1)

In [5]:
# load data
data_path = 'examples/brain_sub.h5ad'
data0 = sc.read(data_path)
data = {'X':torch.from_numpy(data0.X.toarray()).cuda(), 'temp':1e-8, 'properties':None}

/root/anaconda3/envs/flash/lib/python3.9/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [7]:
# inference
vq_code, vq_emb, Cell_rep, h_V_emb, features, gene_indexes, attention = model.encode(data, None, 0, level=12)
gene_attention_weights = attention.mean(1)[:, 0]
Cell_rep, vq_code, vq_emb, features, gene_indexes = map(lambda x:x.detach().cpu().numpy(), [Cell_rep, vq_code, vq_emb, features, gene_indexes])

12


In [ ]:
# umap visualization based on Cell representation
sc_data = sc.AnnData(Cell_rep)
sc.pp.neighbors(sc_data, use_rep='X', random_state=0)
sc.tl.umap(sc_data)
sc_data.obs['true_labels'] = data0.obs['cell_type'].tolist()
sc_data.obs['pred_labels'] =  list(map(str, vq_code.tolist()))
sc.pl.umap(sc_data,color=['true_labels'],wspace=0.3,size=30, frameon=True)
sc.pl.umap(sc_data,color=['pred_labels'],wspace=0.3,size=30, frameon=True)

torch.Size([10000, 201, 201])

In [ ]:
# umap visualization based on quantized embeddings
sc_data = sc.AnnData(vq_emb)
sc.pp.neighbors(sc_data, use_rep='X', random_state=0)
sc.tl.umap(sc_data)
sc_data.obs['pred_labels'] =  list(map(str, vq_code.tolist()))
sc.pl.umap(sc_data,color=['pred_labels'],wspace=0.3,size=30, frameon=True)